In [4]:
import psycopg2

conn_str = "dbname='travel_with_friends' user='Gon' host='localhost'"

conn = psycopg2.connect(conn_str)
cur = conn.cursor()
cur.execute(query)
c = cur.fetchall()

In [2]:
query = """SELECT name, city, state
FROM poi_detail_table_v2
WHERE ST_Distance_Sphere(geom, ST_MakePoint(-122.6659597,45.5083437)) <= 100 * 1609.34;"""

In [8]:
def travel_outside_coords(current_city, current_state, direction=None, n_days=1):
    conn = psycopg2.connect(conn_str)   
    cur = conn.cursor() 
    #coord_long, coord_lat
    cur.execute("select index, coord_lat, coord_long from all_cities_coords_table where city ='%s' and state = '%s';" %(current_city, current_state)) 
    id_, coord_lat, coord_long = cur.fetchone()
    #city, coord_lat, coord_long
    cur.execute("select distinct city, coord_lat, coord_long from all_cities_coords_table where city !='%s' and state = '%s';" %(current_city, current_state))  
    coords = cur.fetchall()    
    
    conn.close()
    
    return id_, coords, coord_lat, coord_long

In [102]:
def direction_from_orgin(start_coord_long,  start_coord_lat, target_coord_long, target_coord_lat):
    angle = calculate_initial_compass_bearing((start_coord_lat, start_coord_long), (target_coord_lat, target_coord_long))
    if (angle > 45) and (angle < 135):
        return 'E'
    elif (angle > 135) and (angle < 215):
        return 'S'
    elif (angle > 215) and (angle < 305):
        return 'W'
    else:
        return 'N'
def check_direction(start_lat, start_long, outside_lat, outside_long, target_direction):
    angle_dict={"E":range(45,135), "S":range(135,215), "W":range(215,305), "N":range(0,45) + range(305,360)}
    angle = calculate_initial_compass_bearing((start_lat, start_long), (outside_lat, outside_long))

    if int(angle) in angle_dict[target_direction]:
        return True
    else: 
        return False

In [99]:
target_direction = "N"
angle_dict={"E":range(45,135), "S":range(135,215), "W":range(215,305), "N":range(0,45) + range(305,360)}

angle = 41.0645080748
# print angle_dict[target_direction]
if int(angle) in angle_dict[target_direction]:
    print angle

41.0645080748


In [104]:
from outside_helpers import *
current_city, current_state = "San Francisco", "California"
a=travel_outside_with_direction(current_city, current_state,"S",140)
print a, len(a)


[(252L, 37.3253815, -121.8629629, 120.0, 8L, 4.5, 371.0), (245L, 37.3318383, -121.9268411, 60.0, 1L, 4.5, 330.0), (246L, 37.3334864, -121.9228839, 120.0, 2L, 4.5, 567.0), (247L, 37.3314363, -121.8902135, 120.0, 3L, 4.0, 685.0), (248L, 37.3183797, -121.950978, 30.0, 4L, 4.0, 1923.0), (249L, 37.3328654, -121.9010368, 180.0, 5L, 4.0, 462.0), (250L, 37.3341062, -121.8904534, 30.0, 6L, 4.5, 195.0), (251L, 37.3268616, -121.8920935, 120.0, 7L, 4.5, 338.0), (253L, 37.3456227, -121.8847222, 60.0, 9L, 4.5, 140.0), (254L, 37.3382082, -121.8863286, 60.0, 10L, 4.5, 136.0), (255L, 37.3053582, -121.8993822, 180.0, 11L, 4.5, 109.0), (256L, 37.2220511, -121.8765117, 60.0, 12L, 4.5, 118.0), (258L, 37.3337953, -121.9127538, 60.0, 14L, 4.5, 75.0), (259L, 37.3211409, -121.8622635, 360.0, 15L, 4.5, 65.0), (260L, 37.3307748, -121.8867089, 60.0, 16L, 5.0, 72.0), (261L, 37.3365213, -121.8943208, 30.0, 17L, 4.0, 99.0), (262L, 37.3336747, -121.8900389, 120.0, 18L, 4.0, 131.0), (263L, 37.3440861, -121.908531, 60.

In [91]:
def travel_outside_with_direction (current_city, current_state, target_direction, furthest_len, n_days=1):
    poi_info = []
    conn = psycopg2.connect(conn_str)   
    cur = conn.cursor() 
    #coord_long, coord_lat
    cur.execute("select index, coord_lat, coord_long from all_cities_coords_table where city ='%s' and state = '%s';" %(current_city, current_state)) 
    id_, start_lat, start_long = cur.fetchone()
    
    cur.execute("SELECT index, coord_lat, coord_long, adjusted_visit_length, ranking, review_score, num_reviews FROM poi_detail_table_v2 WHERE ST_Distance_Sphere(geom, ST_MakePoint(%s,%s)) <= %s * 1609.34;"%(start_long, start_lat,furthest_len)) 
    item = cur.fetchall()
    conn.close()
    for coords in item:
        if check_direction(start_lat, start_long, coords[1] ,coords[2], target_direction):
            poi_info.append(coords)
    return poi_info
    
    


In [30]:
# import city_trip as trip
# # state = 'Virginia'
# # city = 'Newport News'
# state, city = ('Washington', 'Auburn')



# n_days = [1,2,3,4,5]
# for day in n_days:
#     trip.get_fulltrip_data(state, city, int(day))
        

In [3]:
# import city_trip as trip

# for x in range(len(c)):
#     state, city = c[x]
#     if (state == 'Puerto Rico') or (state == 'Virgin Islands'):
#         continue
#     print c[x]
#     n_days = [1,2,3,4,5]
#     for day in n_days:
#         trip.get_fulltrip_data(state, city, int(day))
# not_visited_poi_lst =[]        
# bool(not_visited_poi_lst)

In [4]:
# import psycopg2

# conn_str = "dbname='travel_with_friends' user='Gon' host='localhost'"

# conn = psycopg2.connect(conn_str)
# cur = conn.cursor()
# cur.execute("select max(index) from full_trip_table;")
# full_trip_index = cur.fetchone()[0]
# print full_trip_index
# cur.execute("select county, state from full_trip_table where index='%s';"%(432)) 
# location = cur.fetchone()       
# print location
# cur.execute("select index, county, state, city from county_table where county ='%s' and state='%s';"%(location[0],location[1]))
# index = cur.fetchall()[-1]
# print index
# print index[0]
# conn.close()

In [5]:
# from pyschedule import Scenario, solvers, plotters
# S = Scenario('hello_pyschedule',horizon=30)

# # Create two resources
# Day1, Day2 = S.Resource('Day1'), S.Resource('Day2')

# # Create three tasks with lengths 1,2 and 3
# id1,id2,id3 = S.Task('id1',6), S.Task('id2',12), S.Task('id3',18)

# # Assign tasks to resources, either Alice or Bob
# id1 += Day1
# id2 += Day1|Day2
# id3 += Day1|Day2

# # Solve and print solution
# S.use_makespan_objective()
# solvers.mip.solve(S,msg=1)

# # Print the solution
# print(S.solution())



In [6]:
# import psycopg2
# import city_trip as trip


# conn_str = "dbname='travel_with_friends' user='Gon' host='localhost'"
# state, city = ('Florida', 'Venus')

# conn = psycopg2.connect(conn_str)
# cur = conn.cursor()
# cur.execute("select state, city from county_table where index >24450;")
# # cur.execute("select * from county_table where state = '%s' and city = '%s';"%(state,city))
# c = cur.fetchall()
# print c